#  手書き文字(ひらがな73文字)認識モデルの構築

## 概要
MNIST手書き文字認識で0から9までのラベルを持った10種類の画像の分類を行ったが、更に応用して日本語手書き文字認識モデルを構築していく。構築には2種類のモデルを作成し、ひらがな73文字版の分類と、漢字300文字版の分類の2種類を行う。

> http://lab.ndl.go.jp/cms/hiragana73  
> http://lab.ndl.go.jp/cms/kanji300

In [ ]:
# 画像や前処理周りのimport
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
from tqdm import tqdm_notebook as tqdm
import pandas as pd
%matplotlib inline

from sklearn.model_selection import train_test_split

# 深層学習周りのimport
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# バージョン確認
import matplotlib
import sklearn
import tqdm as tm
print("numpy => {}".format(np.__version__))
print("matplotlib => {}".format(matplotlib.__version__))
print("pandas => {}".format(pd.__version__))
print("OpenCV => {}".format(cv2.__version__))
print("tqdm => {}".format(tm.__version__))
print("scikit-learn => {}".format(sklearn.__version__))
print("keras => {}".format(keras.__version__))

## データセット文字コード対応表の読み込み

In [ ]:
jp_char_df = pd.read_csv("../data/hiragana_table.csv")

In [ ]:
jp_char_df.head()

In [ ]:
# 漢字データセット総数: 146,157
# 1ラベル約100~1000程度
dir_path = "../data/hiragana73"
img_list = []
label_list = []

for root, dirs, files in tqdm(os.walk(dir_path)):
    if len(files) == 0:
        labels = dirs
        labels_dict = dict(zip([_ for _ in range(len(labels))], labels))
    else:
        tmp = []
        idx = jp_char_df[jp_char_df.dir == root.split("/")[-1]].index.values[0]
        for file in files:
            img_list.append(cv2.imread(os.path.join(root, file)))
            label_list.append(idx)

In [ ]:
plt.imshow(img_list[0])

In [ ]:
img_list[0].shape

In [ ]:
len(img_list)

## データの準備

In [ ]:
X, y = np.array(img_list), label_list

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=98, shuffle=True)

X_train = X_train.reshape(len(X_train), X_train[0].shape[0], X_train[0].shape[1], 3)
X_test = X_test.reshape(len(X_test), X_test[0].shape[0], X_test[0].shape[1], 3)

In [ ]:
# trainデータの1枚を確認
plt.imshow(X_train[0], cmap="gray")

In [ ]:
# testデータの1枚を確認
jp_char_df.loc[y_train[0]]["char"]

In [ ]:
print("X_train.shape -> {}".format(X_train.shape))
print("X_test.shape -> {}".format(X_test.shape))

In [ ]:
nb_classes = len(labels)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
print("y_train.shape -> {}".format(len(y_train)))
print("y_test.shape -> {}".format(len(y_test)))

In [ ]:
input_shape = X_train.shape[1:]

## モデルの構築

In [ ]:
model = Sequential()

"""
model.add()の中にConv2DやMaxPooling2Dをいれてモデルを作ってみよう

今回使用する関数一覧

model.add(Dense(次元数, activation=活性化関数))
model.add(Flatten()) # 畳み込みし終えた後全結合層につなげるときに使おう
model.add(Conv2D(次元数, 
                                    kernel_size=(フィルターの縦サイズ, フィルターの横サイズ),
                                    activation=活性化関数,
                                    input_shape=input_shape)) #input_shapeは最初のみ使用
model.add(MaxPooling2D(pool_size=(プーリングの縦サイズ, プーリングの横サイズ))))
model.add(Dropout(0から1までの数値)) # 学習するパーセプトロンのうち使用しない割合を設定

その他、調べてみて便利な関数があればぜひ追加してみよう
"""

# Fully connected layer #2
model.add(Dense(len(labels), activation="softmax"))

In [ ]:
model.summary()

## モデルの読み込み

model = model_from_json(open("../models/kanji_cnn.json", "r").read())
model.load_weights("../params/kanji_cnn_best_weight.hdf5")

## モデルのコンパイル

In [ ]:
"""
モデルを評価する関数をmodel.compile()で定義しよう

実際にmodel.compileの中にはこのようにします

model.compile(loss=誤差関数,
             optimizer=最適化関数,
             metrics=['accuracy']
             )
             
誤差関数
・categorical_crossentropy

最適化関数(好きなものを選ぼう)
・SGD
・Adadelta
・Adam
・RMSprop

評価指標
・accuracy
"""

model.compile(loss="", # 誤差(損失)関数
             optimizer="", # 最適化関数
             metrics=[""] # 評価指標
             )

## 初期モデル・パラメータを保存

In [ ]:
!mkdir ../models

In [ ]:
init_weights_path = '../models/hiragana_cnn_init_weight.hdf5'
best_weights_path = '../models/hiragana_cnn_best_weight.hdf5'
model.save_weights(init_weights_path, overwrite=True)
model.save_weights(best_weights_path, overwrite=True)

In [ ]:
model_path = '../models/hiragana_cnn.json'
model_json = model.to_json()
open(model_path, 'w').write(model_json)

## 学習中のコールバックの設定

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_acc', patience=5,mode='max',verbose=1),
    ModelCheckpoint(best_weights_path,monitor='val_acc', save_best_only=True, 
        mode='max',verbose=0)
]

## 学習

In [ ]:
tic = time.time()

history = model.fit(X_train, Y_train,
                    batch_size=512,
                    epochs=100,
                    verbose=1,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks)

toc = time.time()

print("Execution time: {0:.2f} [sec]".format(toc - tic))

## 学習結果の可視化

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].set_title('Training performance (Loss)')
ax[0].plot(history.epoch, history.history['loss'], label='loss')
ax[0].plot(history.epoch, history.history['val_loss'], label='val_loss')
ax[0].set(xlabel='Epoch', ylabel='Loss')
ax[0].legend()

ax[1].set_title('Training performance (Accuracy)')
ax[1].plot(history.epoch, history.history['acc'], label='acc')
ax[1].plot(history.epoch, history.history['val_acc'], label='val_acc')
ax[1].set(xlabel='Epoch', ylabel='Accuracy')
ax[1].legend(loc='best')

In [ ]:
history_df = pd.DataFrame([history.history["loss"],history.history["acc"],history.history["val_loss"],history.history["val_acc"]])
history_df.index = ["loss", "acc", "val_loss", "val_acc"]

In [ ]:
history_df.T.to_csv("../data/learning_result.csv", index=False)

## 予測の可視化

In [ ]:
Y_test_pred = model.predict(X_test)

In [ ]:
# テストデータの可視化
fig, ax = plt.subplots(1, 10, figsize=(18, 8))

for ii in range(10):
    ax[ii].imshow(X_test[ii].reshape(48, 48), cmap='gray') #iiの値+nでn番目以降のテストデータを出力する．
    ax[ii].axis('off')

In [ ]:
kanji_df = pd.read_csv("../data/kanji_table_prop.csv")

In [ ]:
# 予測の可視化
[kanji_df.loc[y_pred.argmax()]["char"] for y_pred in Y_test_pred[:10]]